In [94]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import time, datetime

In [7]:
data = pd.read_csv('train_ver2.csv/train_ver2.csv')

C:\Users\64161\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [53]:
data = pd.read_csv('test_ver2.csv/test_ver2.csv')

C:\Users\64161\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13647309 entries, 0 to 13647308
Data columns (total 48 columns):
fecha_dato               object
ncodpers                 int64
ind_empleado             object
pais_residencia          object
sexo                     object
age                      object
fecha_alta               object
ind_nuevo                float64
antiguedad               object
indrel                   float64
ult_fec_cli_1t           object
indrel_1mes              object
tiprel_1mes              object
indresi                  object
indext                   object
conyuemp                 object
canal_entrada            object
indfall                  object
tipodom                  float64
cod_prov                 float64
nomprov                  object
ind_actividad_cliente    float64
renta                    float64
segmento                 object
ind_ahor_fin_ult1        int64
ind_aval_fin_ult1        int64
ind_cco_fin_ult1         int64
ind_cder_fin_ult1  

In [30]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [31]:
data = reduce_mem_usage(data)

Memory usage of dataframe is 4997.79 MB
Memory usage after optimization is: 859.42 MB
Decreased by 82.8%


In [71]:
data.columns

Index(['fecha_dato', 'ncodpers', 'ind_empleado', 'pais_residencia', 'sexo',
       'age', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel',
       'ult_fec_cli_1t', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext',
       'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'cod_prov',
       'nomprov', 'ind_actividad_cliente', 'renta', 'segmento'],
      dtype='object')

## data clean

In [54]:
data["fecha_dato"] = pd.to_datetime(data["fecha_dato"],format="%Y-%m-%d")
data["fecha_alta"] = pd.to_datetime(data["fecha_alta"],format="%Y-%m-%d")
data["fecha_dato"].unique()

array(['2016-06-28T00:00:00.000000000'], dtype='datetime64[ns]')

In [55]:
data["month"] = pd.DatetimeIndex(data["fecha_dato"]).month
data["age"]   = pd.to_numeric(data["age"], errors="coerce")
data.loc[data.age < 18,"age"]  = data.loc[(data.age >= 18) & (data.age <= 30),"age"].mean(skipna=True)
data.loc[data.age > 100,"age"] = data.loc[(data.age >= 30) & (data.age <= 100),"age"].mean(skipna=True)
data["age"].fillna(data["age"].mean(),inplace=True)
data["age"] = data["age"].astype(int)

In [56]:
data.loc[data["ind_nuevo"].isnull(),"ind_nuevo"] = 1

In [57]:
data.antiguedad = pd.to_numeric(data.antiguedad,errors="coerce")
data.loc[data.antiguedad.isnull(),"antiguedad"] = data.antiguedad.min()
data.loc[data.antiguedad <0, "antiguedad"] = 0 

In [58]:
dates=data.loc[:,"fecha_alta"].sort_values().reset_index()
median_date = int(np.median(dates.index.values))
data.loc[data.fecha_alta.isnull(),"fecha_alta"] = dates.loc[median_date,"fecha_alta"]
# data["fecha_alta"].describe()

In [59]:
data.loc[data.indrel.isnull(),"indrel"] = 1

In [60]:
data.drop(["tipodom","cod_prov"],axis=1,inplace=True)

In [61]:
data.loc[data.ind_actividad_cliente.isnull(),"ind_actividad_cliente"] = \
data["ind_actividad_cliente"].median()

In [62]:
data.loc[data.nomprov=="CORU\xc3\x91A, A","nomprov"] = "CORUNA, A"
data.loc[data.nomprov.isnull(),"nomprov"] = "UNKNOWN"

In [63]:
data.renta = pd.to_numeric(data.renta, errors="coerce")

In [64]:
incomes = data.loc[data.renta.notnull(),:].groupby("nomprov").agg({"renta":{"MedianIncome":np.median}})
incomes.sort_values(by=("renta","MedianIncome"),inplace=True)
incomes.reset_index(inplace=True)
incomes.nomprov = incomes.nomprov.astype("category", categories=[i for i in data.nomprov.unique()],ordered=False)
# incomes.head()

C:\Users\64161\Anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)
C:\Users\64161\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  after removing the cwd from sys.path.


In [65]:
incomes.head()

,nomprov,renta
,,MedianIncome
0,CIUDAD REAL,62259.930
1,BADAJOZ,62309.280
2,LUGO,64319.265
3,LERIDA,64776.450
4,CASTELLON,66461.640


In [66]:
grouped        = data.groupby("nomprov").agg({"renta":lambda x: x.median(skipna=True)}).reset_index()
new_incomes    = pd.merge(data,grouped,how="inner",on="nomprov").loc[:, ["nomprov","renta_y"]]
new_incomes    = new_incomes.rename(columns={"renta_y":"renta"}).sort_values("renta").sort_values("nomprov")
data.sort_values("nomprov",inplace=True)
data             = data.reset_index()
new_incomes    = new_incomes.reset_index()

In [67]:
data.loc[data.renta.isnull(),"renta"] = new_incomes.loc[data.renta.isnull(),"renta"].reset_index()
data.loc[data.renta.isnull(),"renta"] = data.loc[data.renta.notnull(),"renta"].median()
data.sort_values(by="fecha_dato",inplace=True)

In [68]:
data.loc[data.ind_nomina_ult1.isnull(), "ind_nomina_ult1"] = 0
data.loc[data.ind_nom_pens_ult1.isnull(), "ind_nom_pens_ult1"] = 0

AttributeError: 'DataFrame' object has no attribute 'ind_nomina_ult1'

In [69]:
string_data = data.select_dtypes(include=["object"])
missing_columns = [col for col in string_data if string_data[col].isnull().any()]
for col in missing_columns:
    print("Unique values for {0}:\n{1}\n".format(col,string_data[col].unique()))
del string_data

Unique values for sexo:
['H' 'V' nan]

Unique values for ult_fec_cli_1t:
[nan '2016-06-07' '2016-06-01' '2016-06-28' '2016-06-10' '2016-06-15'
 '2016-06-20' '2016-06-16' '2016-06-21' '2016-06-13' '2016-06-23'
 '2016-06-27' '2016-06-06' '2016-06-22' '2016-06-17' '2016-06-03'
 '2016-06-09' '2016-06-29' '2016-06-02' '2016-06-08' '2016-06-14'
 '2016-06-24']

Unique values for tiprel_1mes:
['I' 'A' 'P' nan]

Unique values for conyuemp:
[nan 'N' 'S']

Unique values for canal_entrada:
['KHE' 'KAT' 'KFA' 'KCC' 'KFC' 'KAG' 'KAW' 'KBZ' 'KHK' 'KCI' 'KBF' 'KHF'
 'KHM' 'KHN' 'KAB' 'KCH' nan 'KAZ' 'KFD' 'KAH' '007' 'RED' 'KAF' 'KHO'
 'KFT' 'KAY' 'KAR' 'KAP' 'KEW' 'KHL' 'KEO' 'KAS' 'KES' 'KEL' 'KAD' 'KHC'
 'KAQ' 'KHQ' 'KBQ' 'KAJ' '013' 'KCF' 'KEV' 'KCG' 'KAE' 'KGX' 'KBM' 'KEZ'
 'KAI' 'KBJ' 'KEM' 'KAA' 'KAO' 'KBH' 'KAK' 'KEY' 'KEJ' 'KCA' 'KCB' 'KCD'
 'KBR' 'KAL' 'KBL' 'KBB' 'KBU' 'KCE' 'KBO' 'KCJ' 'KDM' 'KAM' 'KFM' 'KDR'
 'KAC' 'KBG' 'KHD' 'KEN' 'KCO' 'K00' 'KHS' 'KFU' 'KGV' 'KFJ' 'KFN' 'KFH'
 'KEH' '

In [70]:
data.loc[data.indfall.isnull(),"indfall"] = "N"
data.loc[data.tiprel_1mes.isnull(),"tiprel_1mes"] = "A"
data.tiprel_1mes = data.tiprel_1mes.astype("category")
map_dict = { 1.0  : "1",
            "1.0" : "1",
            "1"   : "1",
            1     : "1",
            2     : "2",
            3     : "3",
            "3.0" : "3",
            "P"   : "P",
            3.0   : "3",
            2.0   : "2",
            "3"   : "3",
            "2.0" : "2",
            4.0   : "4",
            "4.0" : "4",
            "4"   : "4",
            "2"   : "2"}

data.indrel_1mes.fillna("P",inplace=True)
data.indrel_1mes = data.indrel_1mes.apply(lambda x: map_dict.get(x,x))
data.indrel_1mes = data.indrel_1mes.astype("category")
unknown_cols = [col for col in missing_columns if col not in ["indfall","tiprel_1mes","indrel_1mes"]]
for col in unknown_cols:
    data.loc[data[col].isnull(),col] = "UNKNOWN"

In [71]:
feature_cols = data.iloc[:1,].filter(regex="ind_+.*ult.*").columns.values
for col in feature_cols:
    data[col] = data[col].astype(int)

In [72]:
data.isna().sum()

index                    0
fecha_dato               0
ncodpers                 0
ind_empleado             0
pais_residencia          0
sexo                     0
age                      0
fecha_alta               0
ind_nuevo                0
antiguedad               0
indrel                   0
ult_fec_cli_1t           0
indrel_1mes              0
tiprel_1mes              0
indresi                  0
indext                   0
conyuemp                 0
canal_entrada            0
indfall                  0
nomprov                  0
ind_actividad_cliente    0
renta                    0
segmento                 0
month                    0
dtype: int64

In [27]:
product = data.columns[23:-1]

In [91]:
data.iloc[:,23:-1].head()

,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
3193878,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
11981689,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3521925,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8480398,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3521923,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
timeline = pd.to_datetime('2015-12-28',format="%Y-%m-%d")
data_6 = data.loc[data.fecha_dato>=timeline]

In [32]:
pro = data_6.groupby('ncodpers').agg({'ind_ahor_fin_ult1':np.sum}).reset_index()
for i in range(len(product)-1):
    b = data_6.groupby('ncodpers').agg({product[i+1]:np.sum}).reset_index()
    pro = pd.merge(pro,b,on="ncodpers")

In [30]:
pd.DataFrame(product).to_csv("product_index.csv")

In [33]:
mat = np.array(['user','product','times'])
mat_v = []
for i, row in pro.iterrows():
    for j in range(1,pro.shape[1]):
        if row[j] != 0:
            tmp = np.array([row[0],j-1,row[j]])
            mat_v.append(tmp)
            #mat = np.concatenate([mat,tmp],axis=0)
    if i%10000 ==0:
        print(i)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000


In [34]:
mat = np.array(mat_v)
mat = pd.DataFrame(mat)
mat.columns=['ncodpers','item_id','times']
mat.ncodpers = mat.ncodpers.astype('int64')

In [35]:
anti= data_6.groupby('ncodpers').agg({'antiguedad':max}).reset_index()
age = data_6.groupby('ncodpers').agg({'age':max}).reset_index()
ind_n = data_6.groupby('ncodpers').agg({'ind_nuevo':min}).reset_index()
renta = data_6.groupby('ncodpers').agg({'renta':np.mean}).reset_index()

In [36]:
data_6 = data_6.drop(['antiguedad'],1)
data_6n = pd.merge(data_6,anti,how="inner",on="ncodpers")

In [37]:
data_6n = data_6n.drop(['ind_nuevo','renta','age'],1)
data_6n = pd.merge(data_6n,age,how="inner",on="ncodpers")
data_6n = pd.merge(data_6n,ind_n,how="inner",on="ncodpers")
data_6n = pd.merge(data_6n,renta,how="inner",on="ncodpers")

In [38]:
data_6n = pd.concat([data_6n.iloc[:,1:19],data_6n.iloc[:,-4:]],1)

In [39]:
t = pd.to_datetime('2016-05-28',format="%Y-%m-%d")
data_6n = data_6n.loc[data_6n.fecha_dato==t]

In [104]:
data_6n.shape

(931453, 22)

In [40]:
data_6n = data_6n.drop(['fecha_dato'],1)

In [41]:
final = pd.merge(data_6n,mat,on='ncodpers')

In [ ]:
def time_change(data):
    timeArray = time.strptime(data, "%Y-%m-%d")
    timeStamp = int(time.mktime(timeArray))
    return timeStamp
final.fecha_alta = final.fecha_alta.apply(lambda x: time_change(x))

In [42]:
final.head()

,ncodpers,ind_empleado,pais_residencia,sexo,fecha_alta,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,...,indfall,nomprov,ind_actividad_cliente,segmento,antiguedad,age,ind_nuevo,renta,item_id,times
0,307567,N,ES,H,2001-12-14,1.0,UNKNOWN,1,I,S,...,N,MADRID,0.0,02 - PARTICULARES,173.0,44,0.0,82130.55,2,6
1,307567,N,ES,H,2001-12-14,1.0,UNKNOWN,1,I,S,...,N,MADRID,0.0,02 - PARTICULARES,173.0,44,0.0,82130.55,7,6
2,241712,N,ES,V,2001-04-23,1.0,UNKNOWN,1,A,S,...,N,MADRID,1.0,02 - PARTICULARES,181.0,79,0.0,42072.51,2,6
3,241712,N,ES,V,2001-04-23,1.0,UNKNOWN,1,A,S,...,N,MADRID,1.0,02 - PARTICULARES,181.0,79,0.0,42072.51,7,6
4,289461,N,ES,H,2004-11-05,1.0,UNKNOWN,1,A,S,...,N,MADRID,1.0,01 - TOP,132.0,83,0.0,102882.15,2,6


In [44]:
final = final[['ncodpers', 'ind_empleado', 'pais_residencia', 'sexo', 'age',
       'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel', 'ult_fec_cli_1t',
       'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp',
       'canal_entrada', 'indfall', 'nomprov', 'ind_actividad_cliente', 'renta',
       'segmento', 'item_id','times']]

In [45]:
final.to_csv('train.csv',index=None)

In [73]:
data = data.drop(['index','fecha_dato','month'],1)

In [74]:
product.shape[0]

24

In [76]:
test_li = []
k = data.ncodpers
for i in k:
    for j in range(product.shape[0]):
        tmp = np.array([i,j])
        test_li.append(tmp)

In [77]:
test_li = pd.DataFrame(np.array(test_li))

In [78]:
test_li.columns = ['ncodpers','item_id']

In [164]:
test_1.head()

,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,...,indresi,indext,conyuemp,canal_entrada,indfall,nomprov,ind_actividad_cliente,renta,segmento,item_id
0,799826,N,ES,V,50,2008-10-02,0,92,1,UNKNOWN,...,S,N,UNKNOWN,KFC,N,BURGOS,0.0,89738.04,02 - PARTICULARES,ind_ahor_fin_ult1
1,799826,N,ES,V,50,2008-10-02,0,92,1,UNKNOWN,...,S,N,UNKNOWN,KFC,N,BURGOS,0.0,89738.04,02 - PARTICULARES,ind_aval_fin_ult1
2,799826,N,ES,V,50,2008-10-02,0,92,1,UNKNOWN,...,S,N,UNKNOWN,KFC,N,BURGOS,0.0,89738.04,02 - PARTICULARES,ind_cco_fin_ult1
3,799826,N,ES,V,50,2008-10-02,0,92,1,UNKNOWN,...,S,N,UNKNOWN,KFC,N,BURGOS,0.0,89738.04,02 - PARTICULARES,ind_cder_fin_ult1
4,799826,N,ES,V,50,2008-10-02,0,92,1,UNKNOWN,...,S,N,UNKNOWN,KFC,N,BURGOS,0.0,89738.04,02 - PARTICULARES,ind_cno_fin_ult1


In [79]:
test_li_1 = test_li.iloc[:4462160,:]

In [92]:
test_li_1.ncodpers = test_li_1.ncodpers.astype(np.int64)
test_1 = pd.merge(data,test_li_1,on='ncodpers')

In [93]:
test_1.to_csv('test_5.csv',index=None)

In [82]:
test_li_1 = test_li.iloc[4462160:8924320,:]

In [85]:
test_li_1 = test_li.iloc[8924320:13386480,:]

In [88]:
test_li_1 = test_li.iloc[13386480:17848640,:]

In [91]:
test_li_1 = test_li.iloc[17848640:,:]

In [95]:
# import pandas as pd
# data =  pd.read_csv('train.csv',dtype=object)
# test = pd.read_csv('test_1.csv',dtype=object)